# Crash Report Jan 18, 2016

Version: 2016070012

In [1]:
"""
General import statements
"""
import crash_analysis
import pandas as pd
from matplotlib import pyplot as plt 
%matplotlib inline

In [18]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReports\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [11]:
"""
Save/load dataframe to/from CSV
"""
from crash_analysis import read_csv

new_data = False
cache_location = 'src/data/2016070012_jan24_6pm.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = read_csv(cache_location)

In [12]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)
# old num crashes: 672
# Jan 23: 899
# Jan 24: 1157
# Jan 24 6pm: 1284


print(col_names)
print(versions)
print(num_crashes)

['Unnamed: 0', 'ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin16.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'InvParamExpression', 'InvParamFile', 'InvParamFunction', 'InvParamLine', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml', 'empty']
2016070012    1284
201606008      857
Name: AppVersion, dtype: int64
2141


In [13]:
from crash_analysis import filter_dataframe
df = filter_dataframe(df, AppVersion=2016070012)
len(df)

1284

In [14]:
"""
Customer Description Analysis
"""
from crash_analysis.analysis import remove_empty, get_column, stem_frequency
customer_desc = remove_empty(get_column(df, 'ProblemDescription'))

stem_frequency(customer_desc)

print

total words: 1418
tri        	:   70 	 [['trying'], ['trying'], ['trying'], ['trying'], ['trying'], ['trying']]
file       	:   69 	 [['file'], ['file'], ['file'], ['file'], ['file'], ['file']]
return     	:   59 	 [['return'], ['return'], ['return'], ['return'], ['return'], ['return']]
program    	:   48 	 [['program'], ['program'], ['program'], ['program'], ['program'], ['program']]
save       	:   47 	 [['save'], ['saving'], ['saving'], ['saving'], ['saving'], ['save']]
open       	:   45 	 [['opening'], ['opening'], ['opening'], ['opens'], ['opening'], ['opening']]
client     	:   45 	 [['client'], ['client'], ['clients'], ['client'], ['client'], ['clients']]
print      	:   41 	 [['print'], ['print'], ['printing'], ['printing'], ['print'], ['printing']]
crash      	:   30 	 [['crashes'], ['crashes'], ['crashes'], ['crashed'], ['crashed'], ['crashing']]
tax        	:   25 	 [['tax'], ['taxes'], ['tax'], ['tax'], ['tax'], ['tax']]
proseri    	:   25 	 [['proseries'], ['proseries'], 

In [27]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe


def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]


def print_parings(n, field1='Message', groupings=['AppName', 'InstallType', 'OperatingSystem']):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})
    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()

    print("Query {0} \t count: {1} \t {2}: {3}".format(n+1, count, field1, top_key))
    print(field2_hist)
    print(len(field2_hist))
    print '\n'
# 
# pd.set_option('display.max_rows', 5000)
pd.set_option('display.height', 5000)
# [print_parings(n, groupings='ProblemDescription') for n in range(5)]
# print_parings(0, groupings='CustNum')
# [print_parings(n, field1='ExceptionAddress') for n in [0]]
# [print_parings(n, field1='ExceptionAddress', groupings='ProblemDescription') for n in [0]]

[print_parings(n, field1='ExceptionAddress', groupings=['CustNum', 'ProblemDescription']) for n in range(0,1)]

# df[df['CustNum'] == '0614456913']

height has been deprecated.

Query 1 	 count: 44 	 ExceptionAddress: 0x100427a3
CustNum     ProblemDescription                                                                                                                                                                                                                  
0000103064  Nothing                                                                                                                                                                                                                                 1
0000158972  deleting client file and program crashed                                                                                                                                                                                                1
0000619700  deleting non-active clients                                                                                                                                                    

[None]

In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception